In [390]:
import simpy
import random
from scipy.stats import bernoulli

In [391]:
CANTIDAD_CARRILES_GRAN_TONELAJE = 1
CANTIDAD_CARRILES_NORMALES = 2
TIEMPO_LLEGADA = 3 # Llegan cada 3 minutos
TIEMPO_ARREGLO = 2*24*60 # 2 dias x 24 horas x 60 minutos

TIEMPO_SIMULACION = 30*24*60 # 30 dias x 24 horas x 60 minutos (atencion 24/4)
PROBABILIDAD_GRAN_TONELAJE = 0.15 # 15% de que el vehiculo sea de gran tonelaje

REVISION_MEDIA = 20 # Media del tiempo de revision (MINUTOS).
REVISION_DESVIACION_ESTANDAR = 10 # Desviacion estandar de revision (MINUTOS).

PROBABILIDAD_SALIR_SIN_NOTIFICACION = 0.60 # Probabilidad del 60% de salir sin notificaciones.

In [392]:
class Automovil:
    def __init__(self, tipo_automovil: int):
        self.tipo_automovil = tipo_automovil
        self.estado = None
        self.tiempo_arreglado = 0

    def __str__(self):
        return f'tipo: {self.tipo_automovil}, estado {self.estado}'


In [393]:
generar_automovil = lambda : bernoulli.rvs(PROBABILIDAD_GRAN_TONELAJE)

generar_estacionamiento = lambda : random.randint(3, 5)
generar_turno = lambda  : random.randint(1, 3)
generar_matricula = lambda  : random.randint(5, 10)

generar_paso_revision = lambda : random.normalvariate(mu = REVISION_MEDIA, sigma = REVISION_DESVIACION_ESTANDAR)

generar_notificacion = lambda : bernoulli.rvs(PROBABILIDAD_SALIR_SIN_NOTIFICACION)

generar_sello = lambda  : random.randint(1, 3)
generar_salida = lambda  : random.randint(5, 10)

In [394]:
class Emov:
    def __init__(self, environment: simpy.Environment):
        self.environment: simpy.Environment = environment
        self.carril_gran_tonelaje: simpy.Resource = simpy.Resource(environment, CANTIDAD_CARRILES_GRAN_TONELAJE)
        self.carril_normal: simpy.Resource = simpy.Resource(environment, CANTIDAD_CARRILES_NORMALES)
        self.automovil: Automovil = None
        self.automoviles: [dict[int: Automovil]] = []

    def matricular_automovil(self, automovil: Automovil = None):
        self.automovil = automovil

        yield self.environment.process(self.estacionar())
        yield self.environment.process(self.obtener_turno())
        yield self.environment.process(self.pagar_matriculas())
        yield self.environment.process(self.pasar_a_revision())


    def arrivar(self, tipo_automovil):

        self.automovil = Automovil(tipo_automovil=tipo_automovil)

        if self.automovil.tipo_automovil == 0: # Auto normal...
           return self.carril_normal.request()
        else: # Auto gran tonelaje...
            return self.carril_gran_tonelaje.request()

    def estacionar(self):
        yield self.environment.timeout(generar_estacionamiento())

    def obtener_turno(self):
        yield self.environment.timeout(generar_turno())

    def pagar_matriculas(self):
        yield self.environment.timeout(generar_matricula())

    def pasar_a_revision(self):
        if self.automovil is None:
            carril = self.arrivar(generar_automovil())
        else:
            carril = self.arrivar(self.automovil.tipo_automovil)
        if carril.triggered:
            tiempo_revision = generar_paso_revision()
            while tiempo_revision < 0: # Evita que se genere valores negativos para el timeout.
                tiempo_revision = generar_paso_revision()
            yield self.environment.timeout(tiempo_revision)
            carril.resource.release(carril)
            notificacion = generar_notificacion()
            if notificacion == 1: # Sin notificacion
                yield self.environment.process(self.pegar_sello())
                yield self.environment.process(self.salir())
                self.automovil.estado = 'listo'
                self.automoviles.append((self.automovil, self.environment.now))
                self.automovil = None
            else:
                self.automovil.estado = 'renegado'
                self.automovil.tiempo_arreglado = self.environment.now + TIEMPO_ARREGLO
                self.automoviles.append((self.automovil, self.environment.now))

    def pegar_sello(self):
        yield self.environment.timeout(generar_sello())

    def salir(self):
        yield self.environment.timeout(generar_salida())


La empresa desea saber si tiene los recursos necesarios para soportar la demanda de los usuarios e
incrementar el numero de ticktes por día. En base a ello, diseñe y desarrolle un modelo y/o script que
permita realizar la simulación del proceso de pasar cuenca-aire con las siguientes especificaciones
• La empresa tiene 3 carriles de revisión vehicular
• Un solo carril es habilitado para carros de gran tonelaje, busetas, bus, camiones, solo el 15 %
de vehiculos que llegan son de este tipo.
• Cada 3 minutos llega un nuevo vehiculo a revision.
• Se sigue el siguiente proceso
1) Llega el vehiculo al centro, se estaciona (3-5 minutos), saca un turno (1-3 minutos) y pagan
la matricula (5 - 10 minutos).
2) Si esta vacio pasa directo a la revisión que tiene una media de 20 minutos con una
desviacion estandar de 10 minutos.
3) Solo el 60 % de vehiculos pasa sin ninguna notificación, el 40% salen realizan el arreglo (2
dias) y nuevamente realizan el proceso.
4) Despues de pasar la revisión, se les

In [395]:
datos_por_dia = {}

def simular_proceso_matriculacion(global_environment: simpy.Environment):
    global emov
    emov = Emov(environment=global_environment)
    while True:

        if emov.automovil is not None and emov.automovil.tiempo_arreglado > global_environment.now:
            yield environment.process(emov.matricular_automovil(emov.automovil))
        else:
            auto = Automovil(tipo_automovil=generar_automovil())
            yield environment.process(emov.matricular_automovil(auto))

random.seed(42)
emov: Emov = None
environment = simpy.Environment()
environment.process(simular_proceso_matriculacion(environment))
environment.run(until=TIEMPO_SIMULACION)

In [396]:
# Preprocesamiento de los datos

dias_en_minutos = 24*60
segundos = list([x*24*60 for x in range(31)])
segundos.pop(0)

dia_actual = segundos[0]
resultados = dict()
lista = []

for i in emov.automoviles:
    if i[1] <= dia_actual:
        lista.append(i[0].tipo_automovil)
    else:
        resultados[dia_actual] = lista
        dia_actual = segundos[segundos.index(dia_actual)+1]
        lista = list()

## Graficas de resultados

### Numero de vehiculos atendidos al dia por carril

In [397]:
from collections import Counter

pasar_a_dias = lambda x : x/(24*60)
rst = list(map(pasar_a_dias, list(resultados.keys())))

new_resultados = dict(zip(rst, list(resultados.values())))

livianos_por_dia = {f'DIA {int(key)}': Counter(values)[0] for key, values in new_resultados.items()}
pesados_por_dia  = {key: Counter(values)[1] for key, values in new_resultados.items()}

import plotly.graph_objects as go

x_values=list(livianos_por_dia.keys())

fig = go.Figure(data=[
    go.Bar(name='Carril Liviano', x=x_values, y=list(livianos_por_dia.values())),
    go.Bar(name='Carril Pesado',  x=x_values, y=list(pesados_por_dia.values()))
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

### Porcentaje de vehiculos aprovados y no aprovados por dia.

In [398]:
dias_en_minutos = 24*60
segundos = list([x*24*60 for x in range(31)])
segundos.pop(0)

dia_actual = segundos[0]
resultados = dict()
lista = []

for i in emov.automoviles:
    if i[1] <= dia_actual:
        lista.append(i[0].estado)
    else:
        resultados[dia_actual] = lista
        dia_actual = segundos[segundos.index(dia_actual)+1]
        lista = list()

from collections import Counter

pasar_a_dias = lambda x : x/(24*60)
rst = list(map(pasar_a_dias, list(resultados.keys())))

new_resultados = dict(zip(rst, list(resultados.values())))

aprobados_por_dia = {f'DIA {int(key)}': Counter(values).get('listo') for key, values in new_resultados.items()}
rechazados_por_dia  = {key: Counter(values).get('renegado') for key, values in new_resultados.items()}

import plotly.graph_objects as go

x_values=list(livianos_por_dia.keys())

fig = go.Figure(data=[
    go.Bar(name='Aprovados', x=x_values, y=list(aprobados_por_dia.values())),
    go.Bar(name='Rechazado',  x=x_values, y=list(rechazados_por_dia.values()))
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

## Análisis y resultados

### Proceso de desarrollo

- Identificar el problema
- Identificar los procesos que forman parte del proceso de revision.
- Identificar las constantes, tales como, carriles, tiempo empleado en procesos.
- Generar funciones anonimas (lambdas) que permitan generar valores aleatoreos en base a lo especificado en el problema.
- Modelar el problema con simpy.
- Ejecutar el proceso de simulacion.
- Generar graficas.

### Razones para elegir las diferentes distribuciones de probabilidad para generar numeros pseudoaleatoreos.

- Bernoulli: Se utiliza la distribucion de Bernoulli puesto que permite generar valores entre 0 o 1 basado en una determinada probabilidad de exito. En este caso el exito representa diferentes aspectos como la probabilidad de que llegue un vehiculo pesado o un liviano. Tambien se utiliza para determinar la probabilidad de que un determinado vehiculo pase sin ninguna notificacion.

- Random.normalvariate: Se utiliza esta funcion para generar numeros pseudoaleatoreos en base a valores como la Media y la Desviacion Estandar. Fue de utilidad para establecer el tiempo de revision.

### Resultados:

Como resultados se ha podido identificar la simplicidad de Simpy para generar modelos de simulacion en base a problemas de la vida real. Sin embargo, es necesario tener en cuenta que se debe identificar bien el problema, sus variables y la informacion contextual al problema para poder simular correctamente.